In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score
import xgboost as xgb
import time

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../src')
import helpers as h

SEED = 123
holdoutseed = 4
CPU = -1
FOLDS = 5

In [3]:
X_train, X_test, y_train, y_test, \
c_train, c_test, X_holdout, y_holdout, c_holdout, \
features = h.load_data(holdoutseed, engineered_features=False)

#*********** CHANGE TO VER 04 **************
# classifiers = h.load_classifier_dict("classifiers_ver04.pickle")

# Train WITHOUT SMOTE
classifiers = h.load_classifier_dict("classifiers_ver03.pickle")

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [6]:



from sklearn.metrics import f1_score
import numpy as np


params = {}
params['eval_metric'] = "aucpr"
#-------------

max_aucpr = 0
# gridsearch_params = [max_depth for max_depth in range(1,9)]
# gridsearch_params = [min_child_weight for min_child_weight in range(2,10)]
gridsearch_params = [
    (max_depth, eta)
    for max_depth in range(5,6)
    for eta in [.1,.2]]
aucpr_lst = []
# for min_child_weight in gridsearch_params:
for max_depth, eta in gridsearch_params:
    print("CV with max_depth={}, eta={}".format(
                             max_depth,
                             eta))
    # Update our parameters
    params['max_depth'] = max_depth
    params['eta'] = eta

    start_time = time.time()
    cv_results = xgb.cv(params,
                        dtrain,
                        num_boost_round=999,
                        seed=42,
                        nfold=5,
                        metrics={'aucpr'},
                        early_stopping_rounds=10)
    mean_aucpr = cv_results['test-aucpr-mean'].max()
    boost_rounds = cv_results['test-aucpr-mean'].argmax()
    print(f"aucpr {mean_aucpr} for {boost_rounds} rounds")
    if mean_aucpr > max_aucpr:
        max_aucpr = mean_aucpr
        best_params = (max_depth, eta)

    aucpr_lst.append(mean_aucpr)
    t = time.time() - start_time
    print(f"{t:.0f} seconds execution time for max depth {max_depth} eta {eta}")

# print(f"Best params: {best_params}, AUCPR: {max_aucpr}")
print(f"Best params: {best_params[0]}, {best_params[1]}, AUCPR: {max_aucpr}")

CV with max_depth=5, eta=0.1
aucpr 0.8318096 for 42 rounds
53 seconds execution time for max depth 5 eta 0.1
CV with max_depth=5, eta=0.2
aucpr 0.8321072 for 44 rounds
57 seconds execution time for max depth 5 eta 0.2
Best params: 5, 0.2, AUCPR: 0.8321072
